In [1]:
import gc
import shutil
from pathlib import Path

import h5py
import numpy as np
import torch
import tqdm
from datasets import Dataset, Features, Image, Sequence, Value
from PIL import Image as PILImage

from lerobot.common.datasets.lerobot_dataset import CODEBASE_VERSION
from lerobot.common.datasets.push_dataset_to_hub.utils import (
    concatenate_episodes,
    get_default_encoding,
    save_images_concurrently,
)
from lerobot.common.datasets.utils import (
    calculate_episode_data_index,
    hf_transform_to_torch,
)
from lerobot.common.datasets.video_utils import VideoFrame, encode_video_frames
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset 

/home/ns1254/miniforge3/envs/lerobot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# hdf5_path = "/home/ns1254/data_robomimic/nn/lift_image_v141_20p_abs.hdf5"

In [3]:
# f = h5py.File(hdf5_path, "r")
# demos = list(f["data"].keys())

# lengths=[]
# for demo_name in demos:
#     demo=f['data'][demo_name]
#     num_samples=demo.attrs['num_samples']
#     lengths.append(num_samples)

# lengths=np.array(lengths)

# print('Number of demos: ', len(demos))
# print('Max length: ', np.max(lengths))
# print('Min length: ', np.min(lengths))
# print('Mean length: ', np.mean(lengths))

In [4]:
# demo=f['data'][demos[0]]
# actions=demo['actions']
# actions=np.array(actions)
# demo['obs'].keys()

In [5]:
# demo.keys()

In [6]:
# demo_keys=['actions', 'dones', 'rewards', 'states']

In [7]:
# obs_keys = [key for key in demo['obs'].keys()]
# obs_keys

In [8]:
# obs_keys_images=[key for key in obs_keys if 'image' in key]
# obs_keys_others=[key for key in obs_keys if 'image' not in key]
# obs_keys_images, obs_keys_others

In [2]:
def robomimic_to_hf_dataset(data_dict, video) -> Dataset:
    features = {} 

    keys = [key for key in data_dict if "observation.images." in key]
    for key in keys:
        if video:
            features[key] = VideoFrame()
        else:
            features[key] = Image()
            
    for key in data_dict.keys():
        if 'observation.' in key and 'images' not in key:
            features[key] = Sequence(length=data_dict[key].shape[1], feature=Value(dtype="float32", id=None))

    features["action"] = Sequence(length=data_dict["action"].shape[1], feature=Value(dtype="float32", id=None))
    features["episode_index"] = Value(dtype="int64", id=None)
    features["frame_index"] = Value(dtype="int64", id=None)
    features["timestamp"] = Value(dtype="float32", id=None)
    features["next.done"] = Value(dtype="bool", id=None)
    features["index"] = Value(dtype="int64", id=None)
    features["dones"] = Value(dtype="bool", id=None)
    features["rewards"] = Value(dtype="float32", id=None)

    hf_dataset = Dataset.from_dict(data_dict, features=Features(features))
    hf_dataset.set_transform(hf_transform_to_torch)
    return hf_dataset

In [3]:
def load_from_robomimic_hdf5(hdf5_path, videos_dir, fps, video, episodes, encoding):
    f = h5py.File(hdf5_path, "r")
    demos = list(f["data"].keys())

    lengths=[]
    for demo_name in demos:
        demo=f['data'][demo_name]
        num_samples=demo.attrs['num_samples']
        lengths.append(num_samples)

    lengths=np.array(lengths)

    print('Number of demos: ', len(demos))
    print('Max length: ', np.max(lengths))
    print('Min length: ', np.min(lengths))
    print('Mean length: ', np.mean(lengths))

    demo=f['data'][demos[0]]
    demo_keys=['actions', 'dones', 'rewards', 'states']
    obs_keys = [key for key in demo['obs'].keys()]
    obs_keys_images=[key for key in obs_keys if 'image' in key]
    obs_keys_others=[key for key in obs_keys if 'image' not in key]

    num_episodes=len(demos)
     
    ep_dicts = []
    ep_ids = range(num_episodes)
    for ep_idx in tqdm.tqdm(ep_ids):
        demo_name=demos[ep_idx]
        episode=f['data'][demo_name]
        num_frames=episode.attrs['num_samples']  

        ep_dict = {} 

        for key in demo_keys:
            ep_dict[key] = torch.from_numpy(episode[key][:])

        for key in obs_keys_others:
            ep_dict[f"observation.{key}"] = torch.from_numpy(episode['obs'][key][:])

        ep_dict['observation.state'] = ep_dict['states']
        del ep_dict['states']

        ep_dict['action']=ep_dict['actions']
        del ep_dict['actions']

        ep_dict['next.done']=ep_dict['dones']

        for key in obs_keys_images:
            ep_dict[f"observation.images.{key}"] = torch.from_numpy(episode['obs'][key][:])
        
        ep_dict["episode_index"] = torch.tensor([ep_idx] * num_frames)
        ep_dict["frame_index"] = torch.arange(0, num_frames, 1)
        ep_dict["timestamp"] = torch.arange(0, num_frames, 1) / fps 
        # TODO(rcadene): add reward and success by computing them in sim

        assert isinstance(ep_idx, int)
        ep_dicts.append(ep_dict)
    
    data_dict = concatenate_episodes(ep_dicts) 
    total_frames = data_dict["frame_index"].shape[0]
    data_dict["index"] = torch.arange(0, total_frames, 1) 

    return data_dict

In [4]:
def from_robomimic_hdf5_to_lerobot_format(
    hdf5_path: Path,
    videos_dir: Path,
    fps: int | None = None,
    video: bool = True,
    episodes: list[int] | None = None,
    encoding: dict | None = None,
): 
    if fps is None:
        fps = 50
 
    data_dict = load_from_robomimic_hdf5(hdf5_path, videos_dir, fps, video, episodes, encoding)
    hf_dataset = robomimic_to_hf_dataset(data_dict, video=video)
    episode_data_index = calculate_episode_data_index(hf_dataset)
    info = {
        "codebase_version": CODEBASE_VERSION,
        "fps": fps,
        "video": video,
    }  
    return hf_dataset, episode_data_index, info

In [5]:
hdf5_path = Path("/home/ns1254/data_robomimic/nn/lift_image_v141_20p_abs.hdf5")
hf_dataset, episode_data_index, info = from_robomimic_hdf5_to_lerobot_format(hdf5_path, None, 20, False)
print(hf_dataset)
print(episode_data_index)
print(info)

Number of demos:  40
Max length:  63
Min length:  38
Mean length:  49.8


100%|██████████| 40/40 [00:00<00:00, 295.21it/s]


Dataset({
    features: ['dones', 'rewards', 'observation.object', 'observation.robot0_eef_pos', 'observation.robot0_eef_quat', 'observation.robot0_eef_vel_ang', 'observation.robot0_eef_vel_lin', 'observation.robot0_gripper_qpos', 'observation.robot0_gripper_qvel', 'observation.robot0_joint_pos', 'observation.robot0_joint_pos_cos', 'observation.robot0_joint_pos_sin', 'observation.robot0_joint_vel', 'observation.state', 'action', 'next.done', 'observation.images.agentview_image', 'observation.images.robot0_eye_in_hand_image', 'episode_index', 'frame_index', 'timestamp', 'index'],
    num_rows: 1992
})
{'from': tensor([   0,   54,  103,  153,  216,  268,  313,  360,  414,  454,  503,  560,
         608,  651,  692,  740,  789,  830,  883,  930,  977, 1021, 1074, 1112,
        1159, 1201, 1249, 1297, 1344, 1395, 1446, 1498, 1553, 1610, 1669, 1729,
        1779, 1833, 1887, 1942]), 'to': tensor([  54,  103,  153,  216,  268,  313,  360,  414,  454,  503,  560,  608,
         651,  692,  74

In [6]:
dataset=LeRobotDataset.from_preloaded(LeRobotDataset, hf_dataset=hf_dataset, episode_data_index=episode_data_index, info=info) 

In [7]:
dataset

LeRobotDataset(
  Repository ID: '<class 'lerobot.common.datasets.lerobot_dataset.LeRobotDataset'>',
  Split: 'train',
  Number of Samples: 1992,
  Number of Episodes: 40,
  Type: image (.png),
  Recorded Frames per Second: 20,
  Camera Keys: ['observation.images.agentview_image', 'observation.images.robot0_eye_in_hand_image'],
  Video Frame Keys: N/A,
  Transformations: None,
  Codebase Version: v1.6,
)

In [8]:
print(f"\naverage number of frames per episode: {dataset.num_samples / dataset.num_episodes:.3f}")
print(f"frames per second used during data collection: {dataset.fps=}")
print(f"keys to access images from cameras: {dataset.camera_keys=}\n")


average number of frames per episode: 49.800
frames per second used during data collection: dataset.fps=20
keys to access images from cameras: dataset.camera_keys=['observation.images.agentview_image', 'observation.images.robot0_eye_in_hand_image']



In [9]:
from lerobot.common.datasets.populate_dataset import *

In [10]:
stats = compute_stats(dataset)

Compute std: 100%|█████████▉| 248/249 [00:04<00:00, 60.66it/s]


In [12]:
lerobot_dataset=dataset
hf_dataset = lerobot_dataset.hf_dataset
info = lerobot_dataset.info
stats = lerobot_dataset.stats
if stats==None: 
    stats = compute_stats(lerobot_dataset)
    lerobot_dataset.stats = stats

episode_data_index = lerobot_dataset.episode_data_index
# local_dir = lerobot_dataset.videos_dir.parent
local_dir  = Path("/home/ns1254/lerobot/data/lift")
meta_data_dir = local_dir / "meta_data"

hf_dataset = hf_dataset.with_format(None)  # to remove transforms that cant be saved
hf_dataset.save_to_disk(str(local_dir / "train")) 
save_meta_data(info, stats, episode_data_index, meta_data_dir)

Saving the dataset (1/1 shards): 100%|██████████| 1992/1992 [00:00<00:00, 41791.57 examples/s]
/home/ns1254/lerobot/lerobot/scripts/push_dataset_to_hub.py:100: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  episode_data_index = {key: torch.tensor(episode_data_index[key]) for key in episode_data_index}


In [13]:
# load lerobot dataset from disk

from lerobot.common.datasets.lerobot_dataset import LeRobotDataset

local_dir = Path("/home/ns1254/lerobot/data") 
lerobot_dataset = LeRobotDataset(repo_id="lift", root=local_dir)

In [14]:
lerobot_dataset

LeRobotDataset(
  Repository ID: 'lift',
  Split: 'train',
  Number of Samples: 1992,
  Number of Episodes: 40,
  Type: image (.png),
  Recorded Frames per Second: 20,
  Camera Keys: ['observation.images.agentview_image', 'observation.images.robot0_eye_in_hand_image'],
  Video Frame Keys: N/A,
  Transformations: None,
  Codebase Version: v1.6,
)

In [15]:
lerobot_dataset.stats

{'action': {'max': tensor([0.0519, 0.0535, 1.0179, 2.5630, 2.7912, 0.3027, 1.0000]),
  'mean': tensor([-0.0154, -0.0043,  0.8834,  1.9752,  2.1007,  0.0924, -0.4006]),
  'min': tensor([-0.1071, -0.0466,  0.7865, -2.1443, -2.7041, -0.2459, -1.0000]),
  'std': tensor([0.0328, 0.0183, 0.0586, 0.6690, 0.8172, 0.0826, 0.9163])},
 'dones': {'max': tensor([1.]),
  'mean': tensor([0.1009]),
  'min': tensor([0.]),
  'std': tensor([0.3012])},
 'episode_index': {'max': tensor([39.]),
  'mean': tensor([19.7751]),
  'min': tensor([0.]),
  'std': tensor([11.8135])},
 'frame_index': {'max': tensor([62.]),
  'mean': tensor([24.7093]),
  'min': tensor([0.]),
  'std': tensor([14.8983])},
 'index': {'max': tensor([1991.]),
  'mean': tensor([995.4998]),
  'min': tensor([0.]),
  'std': tensor([575.0409])},
 'next.done': {'max': tensor([1.]),
  'mean': tensor([0.1009]),
  'min': tensor([0.]),
  'std': tensor([0.3012])},
 'observation.images.agentview_image': {'max': tensor([[[1.]],
  
          [[1.]],
  
 